In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE=[224,224]
train_path='F:\50       dataset\malaria\Dataset\Train'
valid_path='F:\50       dataset\malaria\Dataset\Test'

In [5]:
vgg19=VGG19(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 338s 4us/step


In [11]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
for layer in vgg19.layers:
    layer.trainable=False

In [9]:
folders=glob('F:/50       dataset/malaria/Dataset/Train/*')

In [10]:
folders

['F:/50       dataset/malaria/Dataset/Train\\Parasite',
 'F:/50       dataset/malaria/Dataset/Train\\Uninfected']

In [13]:
x=Flatten()(vgg19.output)

In [15]:
##for one node or output take sigmoid  for two take softmax
predictions=Dense(len(folders), activation='softmax')(x)
model=Model(inputs=vgg19.input, outputs=predictions)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory('F:\dataset\car_brand pred DL\Train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

In [ ]:
test_set=test_datagen.flow_from_directory('F:\dataset\car_brand pred DL\Test',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

In [ ]:
r=model.fit_generator(
 training_set,
 validation_data=test_set,
 epochs=35,
 steps_per_epoch=len(training_set),
 validation_steps=len(test_set)
                       
)